# Deep learning per Visió per Computador

Emprarem com a framework per realitzar aprenentatge profund la llibreria *keras*. Keras és una capa d'abstrecció que va cridant a les funcions d'un *backend*, típicament *Tensorflow*, que té més llibertat alahora de construir una xarxa però una complexitat molt major. 

**La primera passa i més important és fer ús d'una GPU o una TPU per realitzar l'entrenament.**

## Classificació

Realitzarem un primer experiment amb NN intentant classificar el dataset *MNIST* està format per imatges de caràcters escrits a ma i la seva etiqueta és el número que representen.

El primer que realitzam és la importació dels paquets necessaris de Keras.

In [ ]:
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical

from matplotlib import pyplot as plt

Posterioment cream un el model. Per fer-ho empram l'opció *sequential* que construeix una xarxa com una continum de capes.

In [ ]:
model = models.Sequential()

Afegim capes, en aquest moment només n'afegim de convolucionals i de pooling. Definim la mida de l'entrada. Mostram el resultat de la xarxa al final. 

In [ ]:
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


En aquest moments suposam que la sortida de la darrera capa és un conjunt de característiques que permeten la identifcació, de manera més o manco simple, dels objectes de les imatges. 

Afegim dues capes *FCN* (conegudes també com a denses) com un classificador.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax')) # 10 per que hi ha deu classes

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

### Preparam les dades

Per poder entrenar la xarxa necessitam les dades en la forma que hem indicat abans a la xarxa. *Keras* inclou MNIST com a part de la llibreria. 

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images.shape

(60000, 28, 28)

In [ ]:
train_labels[0]

5

Afegim una dimensió per tal que funciona com a *batch*.

In [ ]:
train_images = train_images.reshape((-1, 28, 28, 1))
test_images = test_images.reshape((-1, 28, 28, 1))

Converim les etiquetes de numèriques a categòriques

In [ ]:
print(train_labels[0])
print(to_categorical(train_labels)[0])

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### Entrenam la xarxa

Per entrenar la xarxa empram les imatges d'entrenament i les seves etiquetes.

In [ ]:
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 4s 4ms/step - loss: 0.3570 - accuracy: 0.9383
Epoch 2/5
 14/938 [..............................] - ETA: 3s - loss: 0.0547 - accuracy: 0.9810

### Test resultats

Per comprovar si els resultats són correctes empram les imatges de test

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

In [ ]:
print(model.predict(test_images[0].reshape((-1, 28, 28, 1))))
plt.imshow(test_images[0][:,:,0]);